# YouTube API

Follow the instructions in the [YouTube API documentation](https://developers.google.com/youtube/v3/getting-started) to create a project and obtain your API key.

The YouTube Data API v3 enforces a quota system to manage rate limits. Each API request costs a certain number of quota units, and every API key is granted 10,000 units per day by default. Here’s how it affects your work:

- Using search.list once costs 100 units.
  - Below, we will search for 5 videos related to “University of Minnesota” using search.list → 100 units
- Each video retrieved with videos.list (with stats/snippet) costs 1 unit.
  - Then, we will retrieve video video stats for each video → 1*5 = 5 units
- Each comment thread retrieved with commentThreads.list costs 1 unit.
  - Retrieve comments per video for 5 videos → 5 units
- Retrieving channel information with channels.list costs 1 unit.

Once you hit your daily quota, you will not be able to make further requests until the next day. You can monitor your quota usage in the Google Cloud Console.

In [ ]:
# Step 1: Install Required Libraries
!pip install --upgrade google-api-python-client

In [ ]:
# Step 2: Set Up API Key
from googleapiclient.discovery import build

API_KEY = ''  # Replace with your API key
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [ ]:
# Step 3: Search for Videos

# First, define your search keyword
query = "University of Minnesota"
# Then: Decide how many results you want
max_results = 5
# Then: Create the API request
request = youtube.search().list( # you are using the search() feature of the YouTube API
    part="snippet",       # We want video details (title, channel, description, etc.)
    q=query,              # The search term
    type="video",         # We're only interested in videos
    maxResults=max_results
)
# In the above code, you can also add in additional parameters like `publishedAfter` to filter by date.
# publishedAfter="2023-01-01T00:00:00Z",
# publishedBefore="2023-12-31T23:59:59Z"

# Then: Execute the request
response = request.execute()

# At last: Print out the video titles
for item in response['items']:
    print(f"Title: {item['snippet']['title']}")
    print(f"Video ID: {item['id']['videoId']}")
    print(f"Channel Title: {item['snippet']['channelTitle']}")
    print("-" * 50)

In [ ]:
# Step 4: See how many subscribers MrBeast has

# MrBeast's channel is https://www.youtube.com/channel/UCX6OQ3DkcsbYNE6H8uQQuVA
# So we can use the channel ID to get his videos
mrbeast_channel_id = 'UCX6OQ3DkcsbYNE6H8uQQuVA'

# Let's first get the uploads playlist ID (where all his videos are stored)
channel_response = youtube.channels().list(
    part="snippet,statistics",     # This tells the API we want channel statistics
    id=mrbeast_channel_id      # MrBeast’s channel
).execute()

# Print out the number of subscribers
print(f"MrBeast has {channel_response['items'][0]['statistics']['subscriberCount']} subscribers.")

In [ ]:
# Step 5: Retrieve MrBeast Video Statistics

# Let's first get the uploads playlist ID (where all his videos are stored)
channel_response = youtube.channels().list(
    part="contentDetails",     # This tells the API we want channel metadata
    id=mrbeast_channel_id      # MrBeast’s channel
).execute()

# Extract the playlist ID for the "uploads" playlist
uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

# Get a few videos from that playlist ID
playlist_response = youtube.playlistItems().list(
    part="snippet",             # We want basic info like title, videoId, etc.
    playlistId=uploads_playlist_id,
    maxResults=5                # Only get 5 videos
).execute()

# Extract video IDs from the playlist response
video_ids = []
for item in playlist_response['items']:
    video_id = item['snippet']['resourceId']['videoId']
    video_ids.append(video_id)

# Retrieve video statistics for these video IDs
video_response = youtube.videos().list(
    part="snippet,statistics",             # We want title, view count, like count, etc.
    id=",".join(video_ids)                 # Join list into one comma-separated string
).execute()

# Print out results for each video
for item in video_response['items']:
    print(f"Title: {item['snippet']['title']}")
    print(f"Views: {item['statistics'].get('viewCount', 'N/A')}")
    print(f"Likes: {item['statistics'].get('likeCount', 'N/A')}")
    print(f"Comments: {item['statistics'].get('commentCount', 'N/A')}")
    print("-" * 50)

In [ ]:
# Step 5: Fetch Comments from a MrBeast Video

# First, let's pick one of MrBeast's video IDs
video_id = video_ids[0]  # This is the first video we got earlier

# Make a request to the YouTube API to get the top-level comments
comments_response = youtube.commentThreads().list(
    part="snippet",              # We want the main comment snippet
    videoId=video_id,            # The video we’re interested in
    maxResults=10,               # Get up to 10 comments
    textFormat="plainText"       # Get raw readable text (not HTML)
).execute()

# Loop through the response and extract the comment text + author
for item in comments_response['items']:
    comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
    author = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
    print(f"{author}: {comment}")
    print("-" * 50)